#### Stance detection with Machine learning

## Introduction
We wanted to detect stance of each quote we had in relation with same-sex marriage. We searched what already existed in the litterature and we found a challenge that took place in 2017 named FakeNewsChallenge where one of the challenges was to find stance detection of a body in relation with a headline.

The algorithm we took was from a UCL team which based their work on the paper *[A simple but tough-to-beat baseline for the Fake News Challenge stance detection task](http://arxiv.org/abs/1707.03264)* from Benjamin Riedel, Isabelle Augenstein, GeorgiosP.Spithourakis and Sebastian Riedel. 
The algorithm takes as input a csv file with headlines and a csv file with bodies and outputs the stance of each quote with 4 different possibilities : agree, disagree, discuss and unrelated 

We had to adapt this model in order to have the best possible results. Indeed, we started by using it with the same headline for all the quotes which gave us mostly unrelated stances. We decided to take multiple headlines and to run the model with the same body for each headline. We then combined the results each run with each headline in order to get better results. 

However, we decided to not use the values we found because there was not enough diversity in our results. 

This file runs the model named pred.py for each headline and we choose manually each headline before launching the model. 


In [6]:
import numpy as np
import pandas as pd
import pred as pred

Creating the constant valies

In [15]:
SIZE_SSM = 32788
headlines_fn = "headlines_final.csv"
temp_headlines_fn = "headlines.csv"
predictions_fn = "final_predictions.csv"
quotes_fn = "data_ssm"
df_headlines = pd.read_csv(headlines_fn)
quotes = "data_ssm.csv"

The next cell gets the different files and creates the dataframes that will be used for the output and the machine learning model. 

In [16]:
df_model_output = pd.read_csv("model_output.csv")
df_output = pd.DataFrame(index=range(32788), columns=['stance'])
h_dataframe = pd.DataFrame(index=range(32788))
h_dataframe['Headline'] = df_headlines['headline'][9]
h_dataframe.index.names = ["Body ID"]
h_dataframe.to_csv(temp_headlines_fn)

We run the main ml model with the inputs and save the results in a csv file that will be used below for the analysis of the outputs. This run will output a dataframe with a column with the stance of the body related to one headline. 

In [17]:
pred.main(temp_headlines_fn, quotes, predictions_fn)
df_pred = pd.read_csv(predictions_fn)
df_pred.head(10)

headlines.csv data_ssm.csv final_predictions.csv


AttributeError: module 'tensorflow' has no attribute 'placeholder'

After getting the stance of each body related to the same headline, we update the model output dataframe with the 4 results by body and we increase the value in the case corresponding to the result. 

In [ ]:
for num, predi in df_pred.iterrows():
    if predi['Stance'] == "agree":
        df_model_output['agree'][num] += 1
    elif predi['Stance'] == "disagree":
        df_model_output['disagree'][num] += 1
    elif predi['Stance'] == "discuss":
        df_model_output['discuss'][num] += 1 
    elif predi['Stance'] == "unrelated":
        df_model_output['unrelated'][num] += 1 
print("ended the first for")  
df_model_output.to_csv(r'model_output.csv')

Finally, we create the last model which will output the value of the dataset by keeping, for each row, the result with the highest value between 'agree', 'disagree', 'discuss' and if for all of these possibilities, the value is 0, then keep unrelated otherwise. The default value in our output will be unrelated.

In [ ]:
df_model_output = pd.read_csv("model_output.csv")
df_output = pd.DataFrame(index=range(32788), columns=['stance'])
for num, row in df_model_output.iterrows():
    if(row.agree != 0 or row.disagree != 0 or row.discuss !=0):
        if(row['agree'] > row['disagree'] and row['agree'] > row['discuss']):
            df_output['stance'][num] = 'agree'
        elif(row['disagree'] > row['agree'] and row['disagree'] > row['discuss']):
            df_output['stance'][num] = 'disagree'
        else:
            df_output['stance'][num] = 'discuss'
    else:
        df_output['stance'][num] = 'unrelated'
df_output.to_csv(r'headlines_final/final_predictions.csv')